In [44]:
import networkx as nx

In [45]:
"""
Abe Lincoln's life in a matriz
"""
A = [[0,  1, 0, 0, 0],
     [0,  0, 1, 0, 0],
     [0,  0, 0, 1, 0],
     [0,  0, 0, 0, 1],
     [.1, 0, 0, 0, 0]]

In [67]:
"""
Constructing a directed unweighted graph with pure Python
"""
from itertools import chain # for flattening the list of edges
G = nx.DiGraph()
edges = chain.from_iterable([(i,j) 
                             for j, column in enumerate(row)
                             if A[i][j]] 
                            for i, row in enumerate(A))
G.add_edges_from(edges)
print(G.edges(data=True))

[(0, 1, {}), (1, 2, {}), (2, 3, {}), (3, 4, {}), (4, 0, {})]


In [68]:
"""
Constructing a directed WEIGHTED graph with pure Python
"""
from itertools import chain # for flattening the list of edges
G = nx.DiGraph()
edges = chain.from_iterable([(i,j,{"weight":A[i][j]})
                             for j, column in enumerate(row)
                             if A[i][j]] 
                            for i, row in enumerate(A))
G.add_edges_from(edges)
print(G.edges(data=True))

[(0, 1, {'weight': 1}), (1, 2, {'weight': 1}), (2, 3, {'weight': 1}), (3, 4, {'weight': 1}), (4, 0, {'weight': 0.1})]


In [69]:
"""
Constructing a directed weighted graph with NumPy
"""
import numpy as np
A_mtx = np.matrix(A)
G = nx.from_numpy_matrix(A_mtx, create_using=nx.DiGraph())
print(G.edges(data=True))

[(0, 1, {'weight': 1.0}), (1, 2, {'weight': 1.0}), (2, 3, {'weight': 1.0}), (3, 4, {'weight': 1.0}), (4, 0, {'weight': 0.1})]


In [70]:
# from graph to adjacency matrix: reverse transformation producing a numPy 2d matrix
B_mtx = nx.to_numpy_matrix(G)
print(B_mtx)

[[ 0.   1.   0.   0.   0. ]
 [ 0.   0.   1.   0.   0. ]
 [ 0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   1. ]
 [ 0.1  0.   0.   0.   0. ]]


In [71]:
print(A_mtx) # checking if some information has been lost in the transformation

[[ 0.   1.   0.   0.   0. ]
 [ 0.   0.   1.   0.   0. ]
 [ 0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   1. ]
 [ 0.1  0.   0.   0.   0. ]]


In [72]:
print(A) # remember that original matrix was a list of list...

[[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0.1, 0, 0, 0, 0]]


In [73]:
B_lst = B_mtx.tolist() # converting a matrix to a list of lists
print(B_lst)

[[0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.1, 0.0, 0.0, 0.0, 0.0]]


In [74]:
"""
Do not understimate the power of Pandas and DataFrame
"""
labels = "Born", "Married", "Elected Rep", "Elected Pres", "Died"
nx.relabel_nodes(G,dict(enumerate(labels)), copy=False)
df = nx.to_pandas_dataframe(G) 
print(df)
print(type(df))

              Born  Married  Elected Rep  Elected Pres  Died
Born           0.0      1.0          0.0           0.0   0.0
Married        0.0      0.0          1.0           0.0   0.0
Elected Rep    0.0      0.0          0.0           1.0   0.0
Elected Pres   0.0      0.0          0.0           0.0   1.0
Died           0.1      0.0          0.0           0.0   0.0
<class 'pandas.core.frame.DataFrame'>


In [75]:
"""
Constructing a directed weighted graph from a DataFrame with Pandas
"""
import pandas as pd
df = pd.DataFrame({
    "from": {0: "Died", 1: "Elected Rep", 2: "Married", 3: "Born", 4: "Elected Pres"},
    "to": {0: "Born", 1: "Elected Pres", 2: "Elected Rep", 3: "Married", 4: "Died"},
    "weight": {0: 0.1, 1: 1, 2: 1, 3: 1, 4: 1},
})
print(df)

           from            to  weight
0          Died          Born     0.1
1   Elected Rep  Elected Pres     1.0
2       Married   Elected Rep     1.0
3          Born       Married     1.0
4  Elected Pres          Died     1.0


In [76]:
G = nx.from_pandas_dataframe(df,"from","to", edge_attr=["weight"], create_using=nx.DiGraph())
print(G.edges(data=True))

[('Died', 'Born', {'weight': 0.1}), ('Born', 'Married', {'weight': 1.0}), ('Elected Rep', 'Elected Pres', {'weight': 1.0}), ('Elected Pres', 'Died', {'weight': 1.0}), ('Married', 'Elected Rep', {'weight': 1.0})]


In [77]:
# we can add other attributes to nodes. 
events = {"Died": 1865, "Born": 1809, "Elected Rep": 1847, "Elected Pres": 1861, "Married": 1842}
nx.set_node_attributes(G,events,"date")
node_data = G.nodes(data=True)
print(node_data)     # note that node_data is a nodeDataView object is created, that is not simply a list a tuples

[('Died', {'date': 1865}), ('Born', {'date': 1809}), ('Elected Rep', {'date': 1847}), ('Elected Pres', {'date': 1861}), ('Married', {'date': 1842})]


In [78]:
# with pandas you may want to work with dataframes instead of a list of tuples. 
lincoln_ser = pd.DataFrame(list(node_data)).set_index(0)[1] # list is necessary under networkx 2.0
print(lincoln_ser)

0
Died            {'date': 1865}
Born            {'date': 1809}
Elected Rep     {'date': 1847}
Elected Pres    {'date': 1861}
Married         {'date': 1842}
Name: 1, dtype: object


In [79]:
# After converting the node labels to the row index, the resulting DataFrame has only one column named 1 
# (which, naturally, is a Series). The values in the column are node attribute dictionaries, and one of 
# the Series constructors builds a Series from a dictionary. Let’s apply the constructor to each row.
# Excerpt From: Dmitry Zinoviev. “Complex Network Analysis in Python" 

df = lincoln_ser.apply(pd.Series)
print(df)

              date
0                 
Died          1865
Born          1809
Elected Rep   1847
Elected Pres  1861
Married       1842


In [80]:
# now you have a DataFrame suitable for processing

spans = df.sort_values('date').diff() # it returns the duration, in years, of each span of Lincoln's life
print(spans)

              date
0                 
Born           NaN
Married       33.0
Elected Rep    5.0
Elected Pres  14.0
Died           4.0


In [81]:
"""
Incidence Matrix allows parallel edges. 
Major drawbacks: 
1. weighted networks cannot be represented
2. a typical complex network takes more memory than an adjacency matrix
Sparse matrices are used to reduce problem 2, but you need to convert it to a dense one to display the content
"""
J = nx.incidence_matrix(G, oriented=True).todense()
print(J)

[[-1.  0.  0.  1.  0.]
 [ 1. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  1.]
 [ 0.  0.  1. -1.  0.]
 [ 0.  1.  0.  0. -1.]]


In [82]:
"""
Edge lists
a list of tuples containing start nod(e, end node, and a dictionary of edge attributes
"""
edges = nx.to_edgelist(G)
print(edges)

[('Died', 'Born', {'weight': 0.1}), ('Born', 'Married', {'weight': 1.0}), ('Elected Rep', 'Elected Pres', {'weight': 1.0}), ('Elected Pres', 'Died', {'weight': 1.0}), ('Married', 'Elected Rep', {'weight': 1.0})]


In [83]:
F = nx.from_edgelist(edges, create_using=nx.DiGraph())
print(F.edges(data=True))

[('Died', 'Born', {'weight': 0.1}), ('Born', 'Married', {'weight': 1.0}), ('Married', 'Elected Rep', {'weight': 1.0}), ('Elected Rep', 'Elected Pres', {'weight': 1.0}), ('Elected Pres', 'Died', {'weight': 1.0})]


In [84]:
print(G.edges(data=True))

[('Died', 'Born', {'weight': 0.1}), ('Born', 'Married', {'weight': 1.0}), ('Elected Rep', 'Elected Pres', {'weight': 1.0}), ('Elected Pres', 'Died', {'weight': 1.0}), ('Married', 'Elected Rep', {'weight': 1.0})]


In [85]:
print(edges)

[('Died', 'Born', {'weight': 0.1}), ('Born', 'Married', {'weight': 1.0}), ('Elected Rep', 'Elected Pres', {'weight': 1.0}), ('Elected Pres', 'Died', {'weight': 1.0}), ('Married', 'Elected Rep', {'weight': 1.0})]


In [86]:
print(nx.is_isomorphic(G,F))

True


In [87]:
"""
Creating a dictionary of lists of nodes
a node is the key, and adjacent nodes are values
Not the best way for recreating the original graph...
"""
dict_list=nx.to_dict_of_lists(G)
print(dict_list)

{'Died': ['Born'], 'Born': ['Married'], 'Elected Rep': ['Elected Pres'], 'Elected Pres': ['Died'], 'Married': ['Elected Rep']}


In [89]:
F = nx.from_dict_of_lists(dict_list,create_using=nx.DiGraph())
print(nx.is_isomorphic(F,G)) # is_isomorphic looks only at the topology, even if the weights are lost
print(G.edges(data=True))
print(F.edges(data=True))

True
[('Died', 'Born', {'weight': 0.1}), ('Born', 'Married', {'weight': 1.0}), ('Elected Rep', 'Elected Pres', {'weight': 1.0}), ('Elected Pres', 'Died', {'weight': 1.0}), ('Married', 'Elected Rep', {'weight': 1.0})]
[('Died', 'Born', {}), ('Born', 'Married', {}), ('Elected Rep', 'Elected Pres', {}), ('Elected Pres', 'Died', {}), ('Married', 'Elected Rep', {})]
